In [2]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
sys.path.append(str(project_root))

In [ ]:
import pandas as pd
from src.valuation.multiples import per_valuation
from src.valuation.graham import graham_number
from src.valuation.dcf import dcf_intrinsic_value
from src.valuation.hybrid import weighted_fair_value
from src.metrics.margin_of_safety import margin_of_safety
from src.data.fetch_price import get_latest_price
from src.data.load_fundamental import load_fundamental

In [ ]:
# Choose ticker symbol
ticker = "AAPL"

In [ ]:
df = load_fundamental(project_root / f"data/raw/{ticker}.csv")
latest = df.sort_values("year").iloc[-1]

In [10]:
fair_value_dcf = dcf_intrinsic_value(
    fcf=latest["free_cash_flow"],
    growth_rate=0.08,
    discount_rate=0.12,
    terminal_growth=0.03,
    years=5,
    shares_outstanding=latest["shares_outstanding"],
)
fair_value_dcf

np.float64(88.83380097090748)

In [5]:
fair_value_per = per_valuation(
    eps=latest["eps"],
    fair_per=15
)
fair_value_per

np.float64(91.95)

In [6]:
fair_value_graham = graham_number(
    eps=latest["eps"],
    book_value_per_share=latest["book_value_per_share"]
)
fair_value_graham

24.211180268627963

In [14]:
values = {
    "dcf": fair_value_dcf,
    "per": fair_value_per,
    "graham": fair_value_graham
}

weights = {
    "dcf": 0.5,
    "per": 0.3,
    "graham": 0.2
}

fair_value = weighted_fair_value(values, weights)
fair_value


np.float64(76.84413653917935)

In [17]:
market_price = get_latest_price("AAPL")

In [ ]:
mos_dcf = margin_of_safety(fair_value_dcf, market_price)
mos_graham = margin_of_safety(fair_value_graham, market_price)
mos_per = margin_of_safety(fair_value_per, market_price)
mos_overall = margin_of_safety(fair_value, market_price)


np.float64(-2.6213563104767856)